# **Red neuronal convolucional para el diagnóstico de nódulos tiroideos según la clasificación EU-TIRADS**

## Por Alejandro Martínez Hernández

### Notebook 2/3

# **Procesamiento de datos**

Ya conociendo la estructura de los datos, estos deben ser preparados y nuevamente organizados para poder usarlos en algún entrenamiento de un modelo. 


## Creación de directorios

En la carpeta "organized" se crean otras carpetas para almacenar las imágenes (\images) y anotaciones (\notes) correspondientes que se van a usar para el entrenamiento.

Dentro de estas carpetas habrán otras que contendrán las imágenes y anotaciones crudas (\Raw) y otra para las imagenes y anotaciones ya procesadas (cropped).

In [1]:
import os
import shutil

def crear_subcarpetas(carpeta_principal, subcarpetas, subcarpeta_padre=None):
    """
    Crea subcarpetas dentro de una carpeta principal. Si se especifica una subcarpeta_padre,
    las subcarpetas se crearán dentro de esta.
    
    Parámetros:
    carpeta_principal (str): Ruta de la carpeta principal.
    subcarpetas (list): Lista de nombres de subcarpetas a crear.
    subcarpeta_padre (str): Nombre de la subcarpeta dentro de la cual se crearán nuevas subcarpetas.
    """
    # Determinar la ruta base donde se crearán las subcarpetas
    ruta_base = carpeta_principal if subcarpeta_padre is None else os.path.join(carpeta_principal, subcarpeta_padre)
    
    # Asegurar que la ruta base exista
    if not os.path.exists(ruta_base):
        os.makedirs(ruta_base)
    
    # Crear cada subcarpeta
    for subcarpeta in subcarpetas:
        os.makedirs(os.path.join(ruta_base, subcarpeta), exist_ok=True)

# Aplicación de función
carpeta_principal_1 = 'db_unal'
carpeta_principal_2 = 'db_unal/organized'
carpeta_principal_3 = 'db_unal/organized/images'
carpeta_principal_4 = 'db_unal/organized/notes'

subcarpetas_dentro_v1 = ['images', 'notes']  
subcarpetas_dentro_v2 = ['raw', 'cropped'] 
subcarpetas_dentro_v3 = ['high', 'medium', 'low']   


# Crear subcarpetas adicionales dentro de db_unal/organized
crear_subcarpetas(carpeta_principal_1, subcarpetas_dentro_v1, 'organized')
crear_subcarpetas(carpeta_principal_2, subcarpetas_dentro_v2, 'images')
crear_subcarpetas(carpeta_principal_3, subcarpetas_dentro_v3, 'raw')
crear_subcarpetas(carpeta_principal_3, subcarpetas_dentro_v3, 'cropped')
crear_subcarpetas(carpeta_principal_4, subcarpetas_dentro_v3, 'raw')
crear_subcarpetas(carpeta_principal_4, subcarpetas_dentro_v3, 'cropped')

## Importar los datos

Se trae el dataframe *df_agrupado.csv* que se generó en el notebook *1_exploracion_datos.ipynb*

In [2]:
import pandas as pd
import os

def cargar_csv_de_forma_segura(ruta_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas de manera segura.
    
    Parámetros:
    ruta_archivo (str): La ruta completa al archivo CSV que se desea cargar.
    
    Retorna:
    DataFrame: Un DataFrame de pandas con los datos cargados desde el archivo CSV.
    None: Retorna None si ocurre un error durante la carga del archivo.
    """
    # Verificar si el archivo existe
    if not os.path.exists(ruta_archivo):
        print(f"El archivo {ruta_archivo} no existe.")
        return None
    
    try:
        # Usar pandas para leer el archivo CSV
        df = pd.read_csv(ruta_archivo)
        return df
    except pd.errors.EmptyDataError:
        print("El archivo está vacío.")
    except pd.errors.ParserError:
        print("Error al parsear el archivo CSV.")
    except Exception as e:
        print(f"Error al cargar el archivo: {e}")
    
    return None

# Cargar el archivo
ruta_archivo = 'df_agrupado.csv'
df_agrupado = cargar_csv_de_forma_segura(ruta_archivo)

# Mostar dataframe
df_agrupado



,Paciente,Edad,Sexo,Numero_imagen,TIRADS,Seg_radiologo_1,Seg_residente_1,Seg_residente_2,Composicion,Ecogenicidad,Margenes,Calcificaciones
0,1,NaN,NaN,1_1,high,1,1,0,NaN,NaN,NaN,NaN
1,1,NaN,NaN,1_2,high,1,1,1,NaN,NaN,NaN,NaN
2,2,49.0,F,2_1,medium,1,1,1,solid,hyperechogenicity,well defined,NaN
3,3,31.0,F,3_1,medium,1,1,1,spongiform,isoechogenicity,well defined,microcalcifications
4,4,37.0,F,4_1,low,1,1,1,spongiform,isoechogenicity,well defined,microcalcifications
...,...,...,...,...,...,...,...,...,...,...,...,...
168,146,NaN,NaN,146_1,low,1,1,0,NaN,NaN,NaN,NaN
169,147,NaN,NaN,147_1,high,1,1,0,NaN,NaN,NaN,NaN
170,148,37.0,F,148_1,medium,1,1,0,solid,hypoechogenicity,well defined,NaN
171,149,78.0,M,149_1,high,1,1,0,NaN,NaN,NaN,NaN


## Copiar datos a tratar

Ya que importamos la base de datos que nos indica que imágenes y anotaciones vamos a usar, se generará una copia de cada uno de estos a las carpetas \raw dentro de \organized.

In [9]:
import pandas as pd
import os
import shutil

ruta_origen = "db_unal/originals/DDTI_V1"
ruta_destino_base = "db_unal/organized/images/raw"

for index, row in df_agrupado.iterrows():
    nombre_imagen = row['Numero_imagen'] + '.jpg'  # Asumiendo que los nombres en el DataFrame no incluyen '.jpg'
    tirads_categoria = row['TIRADS'].lower()  # Asegurándonos de que el nombre del directorio esté en minúsculas

    # Construir la ruta completa del archivo de origen
    archivo_origen = os.path.join(ruta_origen, nombre_imagen)

    # Construir la ruta completa del archivo de destino
    ruta_destino = os.path.join(ruta_destino_base, tirads_categoria, nombre_imagen)

    # Verificar si el archivo existe antes de intentar copiarlo
    if os.path.exists(archivo_origen):
        # Crear el directorio destino si no existe
        os.makedirs(os.path.dirname(ruta_destino), exist_ok=True)
        # Copiar el archivo
        shutil.copy(archivo_origen, ruta_destino)
    else:
        print(f"No se encontró el archivo {archivo_origen}")

No se encontró el archivo db_unal/originals/DDTI_V1\114_1.jpg


## Código para hacer recorte

In [11]:
import cv2
import os
from pathlib import Path

# Definir la ruta base del proyecto
base_path = Path.cwd().parent

# Funciones de ajuste de niveles y recorte de imagen
def ajustar_niveles(imagen, nivel_min, nivel_max):
    """
    Ajusta los niveles de la imagen utilizando cv2.normalize.
    """
    return cv2.normalize(imagen, None, alpha=nivel_min, beta=nivel_max, norm_type=cv2.NORM_MINMAX)

def encontrar_y_recortar_imagen(input_path, output_path):
    """
    Encuentra la imagen de ultrasonido dentro de una captura de pantalla y la recorta
    sin modificar el brillo y el contraste en la imagen recortada.
    """
    # Leer la imagen original
    imagen_original = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
    
    # Convertir la imagen a escala de grises
    imagen_gris = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2GRAY)
    
    # Ajustar los niveles de la imagen para mejorar el contraste
    imagen_contrastada = ajustar_niveles(imagen_gris, 0, 255)
    
    # Umbralizar la imagen para obtener una binarización
    _, imagen_binaria = cv2.threshold(imagen_contrastada, 1, 255, cv2.THRESH_BINARY)
    
    # Encontrar los contornos en la imagen binaria
    contornos, _ = cv2.findContours(imagen_binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Procesar cada contorno encontrado
    for contorno in contornos:
        # Calcular el rectángulo delimitador para el contorno
        x, y, ancho, alto = cv2.boundingRect(contorno)
        borde_mas_corto = min(ancho, alto)
        borde_mas_largo = max(ancho, alto)
        proporcion = borde_mas_largo / borde_mas_corto

        # Comprobar si el contorno cumple con las condiciones de tamaño mínimo y proporción
        if borde_mas_corto >= 250 and proporcion <= 1.5:
            # Recortar la imagen original (sin ajustes de contraste) según el contorno que cumple con los requisitos
            imagen_recortada = imagen_original[y:y+alto, x:x+ancho]
            
            # Guardar la imagen recortada
            cv2.imwrite(output_path, imagen_recortada)
            break  # Terminar después de procesar el primer contorno válido

# Rutas de las carpetas de imágenes usando rutas relativas
input_folder1 = 'db_unal/organized/images/raw'
output_folder1 = 'db_unal/organized/images/cropped'

# Función para procesar las imágenes
def ubicar(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        try:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                input_path = input_folder / filename
                base_name = filename.rsplit('.', 1)[0] + '.png'  # Cambiar la extensión a .png
                output_path = output_folder / base_name
                
                try:
                    encontrar_y_recortar_imagen(str(input_path), str(output_path))
                except Exception as e:
                    print(f"No se pudo procesar el archivo {filename}: {e}")
        except Exception as e:
            print(f"No se pudo procesar el archivo {filename}: {e}")

# Procesar imágenes
ubicar(input_folder1, output_folder1)

No se pudo procesar el archivo 100_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 101_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 102_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 103_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 104_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 105_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 106_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 107_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 108_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 109_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 10_1.jpg: unsupported operand type(s) f

## Código para hacer undersampling

##  Aumento de datos

Dejaré este código para volver a usarlo en un futuro

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Definir la ruta base del proyecto
base_path = Path.cwd().parent

# Ubicación de las imágenes ordenadas (usando pathlib para construir la ruta)
database = base_path / 'data' / 'us_images' / 'procesadas' / 'ordenadas'

# Instanciar ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=15,
    zoom_range=[0.5, 1.5],
    validation_split=0.2
)

# Generadores para sets de entrenamiento y validación
data_gen_entrenamiento = datagen.flow_from_directory(
    database,
    target_size=(300, 300),
    batch_size=32,
    shuffle=True,
    subset='training'
)

data_gen_pruebas = datagen.flow_from_directory(
    database,
    target_size=(300, 300),
    batch_size=32,
    shuffle=True,
    subset='validation'
)

# Imprimir 10 imágenes del generador de entrenamiento
for imagen, etiqueta in data_gen_entrenamiento:
    for i in range(10):
        plt.subplot(2, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(imagen[i])
    break
plt.show()